In [2]:
import pickle
import numpy as np

In [3]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        return pickle.dump(obj, f)

In [92]:
l = np.array([i.lower().replace(" ","") for i in sorted(load_obj("pokemon_attacks_classifier/Models/Classify_Attacks/kanto_attacks_list"))])
attack_map = load_obj("pokemon_attacks_classifier/Models/Classify_Attacks/pokemon_attack_map")

In [93]:
print(attack_map["Charmander"])

{'leer', 'ember', 'rage', 'slash', 'growl', 'flamethrower', 'fire spin', 'scratch'}


In [94]:
# generate pokemon attack pairs
pokemon_attack_pairs = [" ".join([pokemon, attack])for pokemon in attack_map for attack in attack_map[pokemon]]
print(len(pokemon_attack_pairs))

1009


In [95]:
base_url = "https://archives.bulbagarden.net/wiki/Special:ListFiles?limit=100000&ilshowall=1&user=&ilsearch="
query_urls = [base_url+(pair.lower()).replace(" ", "+") for pair in pokemon_attack_pairs]
print(query_urls[2])

https://archives.bulbagarden.net/wiki/Special:ListFiles?limit=100000&ilshowall=1&user=&ilsearch=parasect+growth


In [7]:
# Try 1
# import requests
# import multiprocessing

# def internet_resource_getter(urls):
#     session = requests.Session()
#     stuff_got = []
#     for url in urls:
#         response = session.get(str(url))
#         stuff_got.append(response.json())

#     return stuff_got

# pool = multiprocessing.Pool(processes=3)
# pool_outputs = pool.map(internet_resource_getter, query_urls)
# pool.close()
# pool.join()
# print(pool_outputs)

In [8]:
# Try 3
# from requests_threads import AsyncSession

# session = AsyncSession(n=100)

# async def _main():
#     rs = []
#     for url in query_urls[:10]:
#         rs.append(await session.get(url))
#     print(rs)

# if __name__ == '__main__':
#     session.run(_main)

In [9]:
# Try 2
# import grequests
# rs = (grequests.get(u) for u in query_urls)
# responses = grequests.map(rs)

In [134]:
# Try 2 with Retry
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
s = requests.Session()
retries = Retry(total=5, backoff_factor=0.2, status_forcelist=[500, 502, 503, 504], raise_on_redirect=True,
                raise_on_status=True)
s.mount('http://', HTTPAdapter(max_retries=retries))
s.mount('https://', HTTPAdapter(max_retries=retries))
reqs = (grequests.get(url, session=s, headers=header) for url in query_urls)
responses = grequests.map(reqs)

pair_all_links = []
for resp in responses:
    temp = []
    soup = BeautifulSoup(resp.text)
    for link in soup.findAll('a', href=True, text='file'):
        temp.append("https:"+link["href"])
    pair_all_links.append(temp)
#     print("https:"+link["href"])

In [135]:
print(sum([len(i) for i in pair_all_links if i]))

1511


In [35]:
# import csv
# r_htmls = []
# for r in responses:
#     if r:
#         r_htmls.append(r.text)
#     else:
#         r_htmls.append(None)
# with open("bulbapedia_res.csv", 'w', newline='\n') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(r_htmls)

In [34]:
# responses[0].text

In [36]:
# from bs4 import BeautifulSoup

# all_links = []
# for response in responses:
#     pair_wise_links = []
#     if response:
#         soup = BeautifulSoup(response.text)
#         for link in soup.findAll('a', href=True, text='file'):
#             pair_wise_links.append("https:"+link["href"])
#     all_links.append(pair_wise_links)

In [136]:
pair_all_links[0]

['https://archives.bulbagarden.net/media/upload/d/d3/Cassandra_Parasect_Spore.png',
 'https://archives.bulbagarden.net/media/upload/d/d3/Cassandra_Parasect_Spore.png']

In [137]:
temp_pairs = [(pokemon, attack) for pokemon in attack_map for attack in attack_map[pokemon]]
pokemon_attack_urls_map = [{"pokemon": pokemon, "attack": attack, "bulba_url": bulba_url, "urls": urls} for (pokemon, attack), bulba_url, urls in zip(temp_pairs, query_urls, pair_all_links)]

In [140]:
print(pokemon_attack_urls_map[0])
save_obj(pokemon_attack_urls_map,"data/scraped_urls/pokemon_attack_urls_map")

{'pokemon': 'Parasect', 'attack': 'spore', 'bulba_url': 'https://archives.bulbagarden.net/wiki/Special:ListFiles?limit=100000&ilshowall=1&user=&ilsearch=parasect+spore', 'urls': ['https://archives.bulbagarden.net/media/upload/d/d3/Cassandra_Parasect_Spore.png', 'https://archives.bulbagarden.net/media/upload/d/d3/Cassandra_Parasect_Spore.png']}


In [141]:
# to see all bulba_urls with no data in it
# print([i["bulba_url"] for i in pokemon_attack_urls_map if not i["urls"]])

In [5]:
# to count all urls
pokemon_attack_urls_map = load_obj("data/scraped_urls/pokemon_attack_urls_map")
temp = [i["urls"] for i in pokemon_attack_urls_map if i["urls"]]
print(len([i for t in temp for i in t]))

1511


In [6]:
import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36')]
urllib.request.install_opener(opener)

image_id = 10000
pairs_to_image_map = list()
for data in pokemon_attack_urls_map:
    for url in data["urls"]:
        urllib.request.urlretrieve(url, "battle_scenes_data/" + str(image_id) + ".jpg")
        pairs_to_image_map.append({"pokemon": data["pokemon"], "attack": data["attack"], "image_id": image_id})
        image_id+=1
# save_obj(pairs_to_image_map,"battle_scenes_data/pairs_to_image_map")

In [ ]:
# convert RGBA to RGB
import os
from PIL import Image
base_path = "battle_scenes_data/battle_scenes/"

for file in os.listdir(base_path):
    if ".jpg" in file:
        png = Image.open(base_path + file)
        png.load() # required for png.split()
        if len(png.split()) == 4:
            background = Image.new("RGB", png.size, (255, 255, 255))
            background.paste(png, mask=png.split()[3]) # 3 is the alpha channel

            background.save(base_path + file, 'JPEG', quality=80)
            print(file)

In [72]:
# now try for only attacks instead of pairs
attacks = [i for i in sorted(load_obj("pokemon_attacks_classifier/Models/Classify_Attacks/kanto_attacks_list"))]
base_url = "https://archives.bulbagarden.net/wiki/Special:ListFiles?limit=100000&ilshowall=1&user=&ilsearch="
attack_query_urls = [base_url+(attack.lower()).replace(" ", "+") for attack in attacks]
print(attack_query_urls[5])

https://archives.bulbagarden.net/wiki/Special:ListFiles?limit=100000&ilshowall=1&user=&ilsearch=aurora+beam


In [86]:
# import grequests
# attack_rs = (grequests.get(u) for u in attack_query_urls)
# attack_responses = grequests.map(attack_rs)

In [114]:
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

s = requests.Session()
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

retries = Retry(total=5, backoff_factor=0.2, status_forcelist=[500, 502, 503, 504], raise_on_redirect=True,
                raise_on_status=True)
s.mount('http://', HTTPAdapter(max_retries=retries))
s.mount('https://', HTTPAdapter(max_retries=retries))

reqs = (grequests.get(url, session=s, headers=header) for url in attack_query_urls)

all_links = []
for response in grequests.imap(reqs, stream=False):
    pair_wise_links = []
#     response = requests.get(url, headers=header)
    if response:
        soup = BeautifulSoup(response.text)
        for link in soup.findAll('a', href=True, text='file'):
            pair_wise_links.append("https:"+link["href"])
    all_links.append(pair_wise_links)
    if not pair_wise_links:
        print("Fail")
        print(url)

In [115]:
print(all_links[1][0])

https://archives.bulbagarden.net/media/upload/d/d1/Gastro_Acid_PMD_GTI.png


In [119]:
attack_urls_map = [{"attack": attack, "bulba_url": bulba_url, "urls": urls} for attack, bulba_url, urls in zip(attacks, attack_query_urls, all_links)]
save_obj(attack_urls_map,"data/scraped_urls/attack_urls_map")

In [120]:
attack_urls_map = load_obj("data/scraped_urls/attack_urls_map")
attack_urls_map[0]["urls"][0]

'https://archives.bulbagarden.net/media/upload/7/71/Absorb_PMD_GTI.png'

In [122]:
# to see all bulba_urls with no data in it
print(sum([len(i["urls"]) for i in attack_urls_map if i["urls"]]))

14160


In [ ]:
# not downloaded the attacks only dataset